In [138]:
import pandas as pd
import MetaTrader5 as mt5
from datetime import datetime, timedelta
from pandas_market_calendars import get_calendar
from workadays import workdays as wd

In [126]:
class Setups():
    def __init__(self, ativos, timeframe):
        self.mt5 = mt5
        self.data_inicio = datetime.now() - timedelta(weeks=53*5)
        self.data_fim = datetime.now()
        self.ativos = ativos
        self.timeframe = timeframe
        self.cotacoes = []
        

    def historico_cotacoes(self):
        '''Ticker: str or list
            Timeframe: D -> Daily, 60m -> 60 minutes, 15m -> 15 minutes, 5m -> 5 minutes, 1m -> 1 minute
        '''

        if self.timeframe == 'D':
            timeframe = mt5.TIMEFRAME_D1
        elif self.timeframe == '60m':
                timeframe = mt5.TIMEFRAME_H1
        elif self.timeframe == '15m':
            timeframe = mt5.TIMEFRAME_M15
        elif self.timeframe == '5m':
            timeframe = mt5.TIMEFRAME_M5
        elif self.timeframe == '1m':
            timeframe = mt5.TIMEFRAME_M1

        for ativo in self.ativos:
            try:
                cotacoes_ativo = mt5.copy_rates_from(ativo, timeframe, self.data_fim, 150000)
                cotacoes_ativo = pd.DataFrame(cotacoes_ativo)
                cotacoes_ativo['ticker'] = str(ativo)
                cotacoes_ativo.index = pd.to_datetime(cotacoes_ativo['time'], unit='s')
                cotacoes_ativo = cotacoes_ativo.drop(columns='time')
                self.cotacoes.append(cotacoes_ativo)
                
            except Exception as e:
                print(f'Erro ao encontrar dados para {ativo}: {e}')
        
        return self.cotacoes
        


In [127]:
ativos = ['WDO$']
timeframe = '5m'


In [128]:
tester = Setups(ativos, timeframe)

In [129]:
tester.historico_cotacoes()

[                         open      high       low     close  tick_volume  \
 time                                                                       
 2019-03-08 09:00:00  4848.726  4865.627  4848.726  4853.733         8722   
 2019-03-08 09:05:00  4853.733  4858.741  4849.352  4853.733         6433   
 2019-03-08 09:10:00  4854.359  4856.237  4844.970  4848.100         9030   
 2019-03-08 09:15:00  4848.726  4859.367  4847.474  4856.237         6088   
 2019-03-08 09:20:00  4856.237  4859.367  4852.481  4853.733         4203   
 ...                       ...       ...       ...       ...          ...   
 2024-03-08 09:55:00  4970.500  4972.000  4970.000  4971.500          804   
 2024-03-08 10:00:00  4971.500  4976.500  4970.000  4975.000         2048   
 2024-03-08 10:05:00  4975.000  4977.000  4973.000  4973.500         2002   
 2024-03-08 10:10:00  4973.500  4984.000  4973.500  4984.000         3056   
 2024-03-08 10:15:00  4984.000  4984.500  4982.500  4983.500         1294   

In [130]:
ativo = 'WDO$'
timeframe = mt5.TIMEFRAME_M5
data_fim = datetime.now()
print(data_fim)

2024-03-08 10:16:44.817191


In [133]:
mt5.initialize()
cotacoes = mt5.copy_rates_from(ativo, timeframe, data_fim, 150000)
cotacoes = pd.DataFrame(cotacoes)
cotacoes['ticker'] = str(ativo)
cotacoes.index = pd.to_datetime(cotacoes['time'], unit='s')
cotacoes = cotacoes.drop(columns='time')
cotacoes

,open,high,low,close,tick_volume,spread,real_volume,ticker
time,,,,,,,,
2019-03-08 09:00:00,4848.726,4865.627,4848.726,4853.733,8722,1,24327,WDO$
2019-03-08 09:05:00,4853.733,4858.741,4849.352,4853.733,6433,1,18409,WDO$
2019-03-08 09:10:00,4854.359,4856.237,4844.970,4848.100,9030,1,30850,WDO$
2019-03-08 09:15:00,4848.726,4859.367,4847.474,4856.237,6088,1,18322,WDO$
2019-03-08 09:20:00,4856.237,4859.367,4852.481,4853.733,4203,1,13385,WDO$
...,...,...,...,...,...,...,...,...
2024-03-08 10:15:00,4984.000,4986.000,4982.500,4985.500,3670,0,63555,WDO$
2024-03-08 10:20:00,4986.000,4989.000,4984.500,4986.500,3328,0,50407,WDO$
2024-03-08 10:25:00,4987.000,4988.000,4982.500,4985.000,3193,0,38799,WDO$


In [135]:
# Cria um calendário de mercado para o NYSE (New York Stock Exchange)
nyse = get_calendar('NYSE')

# Define o intervalo de datas dos últimos 5 anos
start_date = pd.Timestamp.now() - pd.DateOffset(years=5)
end_date = pd.Timestamp.now()

# Obtém os dias de payroll (dias úteis) para o intervalo de datas especificado
payroll_days = nyse.valid_days(start_date, end_date)

# Converte os dias de payroll em um DataFrame do pandas para facilitar a manipulação
payroll_days_df = pd.DataFrame(payroll_days, columns=['Date'])

# Exibe os dias de payroll
print(payroll_days_df)

                          Date
0    2019-03-08 00:00:00+00:00
1    2019-03-11 00:00:00+00:00
2    2019-03-12 00:00:00+00:00
3    2019-03-13 00:00:00+00:00
4    2019-03-14 00:00:00+00:00
...                        ...
1255 2024-03-04 00:00:00+00:00
1256 2024-03-05 00:00:00+00:00
1257 2024-03-06 00:00:00+00:00
1258 2024-03-07 00:00:00+00:00
1259 2024-03-08 00:00:00+00:00

[1260 rows x 1 columns]


In [143]:
def payroll_dias():
    mes_atual = datetime.now().month
    ano_atual = datetime.now().year
    ano_ini = ano_atual - 5
     
    payrolls = []
    # Criar um objeto de feriados para o Brasil
    feriados = wd.get_holidays(country='US', years=range(ano_ini, ano_atual))
    
    for ano in range(ano_ini, ano_atual + 1):
        for mes in range(1, 13):
            # Obter o primeiro dia do mês
            primeiro_dia = datetime(ano, mes, 1).date()
            # Encontrar a sexta-feira do mês
            sexta_feira = primeiro_dia + timedelta(days=(4 - primeiro_dia.weekday()) % 7)
            # Contador para a terceira sexta-feira útil
            contador_sexta = 0
            while sexta_feira.month == mes:
                # Verificar se a sexta-feira é um feriado brasileiro
                if sexta_feira in feriados:
                    # Se for um feriado, retornar o dia útil anterior
                    sexta_feira = sexta_feira + timedelta(days=7)
                    while sexta_feira.weekday() >= 5: # Enquanto for fim de semana
                        sexta_feira += timedelta(days=1)
                # Incrementar o contador e adicionar a sexta-feira útil
                contador_sexta += 1
                if contador_sexta == 1:
                    payrolls.append(sexta_feira)
                    break # Sair do loop quando a terceira sexta-feira útil for encontrada
                sexta_feira += timedelta(days=7)
    
    
    vencimentos = [data.strftime('%Y-%m-%d') for data in payrolls]
    
    return vencimentos

In [152]:
payroll_espec = ['2019-03-08', '2020-01-10', '2020-05-08', '2021-10-08', '2022-07-08', '2023-03-10', '2023-12-08', '2024-03-08']

In [154]:
payrolls = payroll_dias()
len(payrolls)

72

In [167]:
payroll_espec[0]

'2019-03-08'

In [168]:
payrolls_total = []
for i in payroll_espec:
    payrolls_total.append(payroll_espec[i])
payrolls_total

TypeError: list indices must be integers or slices, not str

In [155]:
mapa_meses = {data.split('-')[1]: data for data in payroll_espec}

# Lista para armazenar as datas atualizadas
lista_atualizada = []

# Itera sobre a lista principal e substitui as datas conforme necessário
for data in payrolls:
    mes = data.split('-')[1]
    if mes in mapa_meses:
        lista_atualizada.append(mapa_meses[mes])
    else:
        lista_atualizada.append(data)

# Exibe a lista atualizada
len(lista_atualizada)

72

In [161]:
mapa_meses = {data.split('-')[1]: data for data in payroll_espec}

lista_principal_datetime = [datetime.strptime(data, '%Y-%m-%d').date() for data in payrolls]

# Itera sobre a lista de objetos datetime e substitui as datas conforme necessário
for i, data in enumerate(lista_principal_datetime):
    mes = data.strftime('%m')
    if mes in mapa_meses:
        lista_principal_datetime[i] = datetime.strptime(mapa_meses[mes], '%Y-%m-%d').date()

lista_principal_datetime

[datetime.date(2020, 1, 10),
 datetime.date(2019, 2, 1),
 datetime.date(2024, 3, 8),
 datetime.date(2019, 4, 5),
 datetime.date(2020, 5, 8),
 datetime.date(2019, 6, 7),
 datetime.date(2022, 7, 8),
 datetime.date(2019, 8, 2),
 datetime.date(2019, 9, 6),
 datetime.date(2021, 10, 8),
 datetime.date(2019, 11, 1),
 datetime.date(2023, 12, 8),
 datetime.date(2020, 1, 10),
 datetime.date(2020, 2, 7),
 datetime.date(2024, 3, 8),
 datetime.date(2020, 4, 3),
 datetime.date(2020, 5, 8),
 datetime.date(2020, 6, 5),
 datetime.date(2022, 7, 8),
 datetime.date(2020, 8, 7),
 datetime.date(2020, 9, 4),
 datetime.date(2021, 10, 8),
 datetime.date(2020, 11, 6),
 datetime.date(2023, 12, 8),
 datetime.date(2020, 1, 10),
 datetime.date(2021, 2, 5),
 datetime.date(2024, 3, 8),
 datetime.date(2021, 4, 2),
 datetime.date(2020, 5, 8),
 datetime.date(2021, 6, 4),
 datetime.date(2022, 7, 8),
 datetime.date(2021, 8, 6),
 datetime.date(2021, 9, 3),
 datetime.date(2021, 10, 8),
 datetime.date(2021, 11, 5),
 datetime

In [162]:
sorted(lista_principal_datetime)

[datetime.date(2019, 2, 1),
 datetime.date(2019, 4, 5),
 datetime.date(2019, 6, 7),
 datetime.date(2019, 8, 2),
 datetime.date(2019, 9, 6),
 datetime.date(2019, 11, 1),
 datetime.date(2020, 1, 10),
 datetime.date(2020, 1, 10),
 datetime.date(2020, 1, 10),
 datetime.date(2020, 1, 10),
 datetime.date(2020, 1, 10),
 datetime.date(2020, 1, 10),
 datetime.date(2020, 2, 7),
 datetime.date(2020, 4, 3),
 datetime.date(2020, 5, 8),
 datetime.date(2020, 5, 8),
 datetime.date(2020, 5, 8),
 datetime.date(2020, 5, 8),
 datetime.date(2020, 5, 8),
 datetime.date(2020, 5, 8),
 datetime.date(2020, 6, 5),
 datetime.date(2020, 8, 7),
 datetime.date(2020, 9, 4),
 datetime.date(2020, 11, 6),
 datetime.date(2021, 2, 5),
 datetime.date(2021, 4, 2),
 datetime.date(2021, 6, 4),
 datetime.date(2021, 8, 6),
 datetime.date(2021, 9, 3),
 datetime.date(2021, 10, 8),
 datetime.date(2021, 10, 8),
 datetime.date(2021, 10, 8),
 datetime.date(2021, 10, 8),
 datetime.date(2021, 10, 8),
 datetime.date(2021, 10, 8),
 datet

In [156]:
lista_atualizada

['2020-01-10',
 '2019-02-01',
 '2024-03-08',
 '2019-04-05',
 '2020-05-08',
 '2019-06-07',
 '2022-07-08',
 '2019-08-02',
 '2019-09-06',
 '2021-10-08',
 '2019-11-01',
 '2023-12-08',
 '2020-01-10',
 '2020-02-07',
 '2024-03-08',
 '2020-04-03',
 '2020-05-08',
 '2020-06-05',
 '2022-07-08',
 '2020-08-07',
 '2020-09-04',
 '2021-10-08',
 '2020-11-06',
 '2023-12-08',
 '2020-01-10',
 '2021-02-05',
 '2024-03-08',
 '2021-04-02',
 '2020-05-08',
 '2021-06-04',
 '2022-07-08',
 '2021-08-06',
 '2021-09-03',
 '2021-10-08',
 '2021-11-05',
 '2023-12-08',
 '2020-01-10',
 '2022-02-04',
 '2024-03-08',
 '2022-04-01',
 '2020-05-08',
 '2022-06-03',
 '2022-07-08',
 '2022-08-05',
 '2022-09-02',
 '2021-10-08',
 '2022-11-04',
 '2023-12-08',
 '2020-01-10',
 '2023-02-03',
 '2024-03-08',
 '2023-04-07',
 '2020-05-08',
 '2023-06-02',
 '2022-07-08',
 '2023-08-04',
 '2023-09-01',
 '2021-10-08',
 '2023-11-03',
 '2023-12-08',
 '2020-01-10',
 '2024-02-02',
 '2024-03-08',
 '2024-04-05',
 '2020-05-08',
 '2024-06-07',
 '2022-07-

In [150]:
lista_atualizada = sorted([datetime.strptime(data, '%Y-%m-%d') for data in lista_atualizada])

# Converte os objetos datetime de volta para strings no formato original
lista_atualizada = [data.strftime('%Y-%m-%d') for data in lista_atualizada]
lista_atualizada


['2019-02-01',
 '2019-04-05',
 '2019-06-07',
 '2019-08-02',
 '2019-09-06',
 '2019-11-01',
 '2020-01-10',
 '2020-01-10',
 '2020-01-10',
 '2020-01-10',
 '2020-01-10',
 '2020-01-10',
 '2020-02-07',
 '2020-04-03',
 '2020-05-08',
 '2020-05-08',
 '2020-05-08',
 '2020-05-08',
 '2020-05-08',
 '2020-05-08',
 '2020-06-05',
 '2020-08-07',
 '2020-09-04',
 '2020-11-06',
 '2021-02-05',
 '2021-04-02',
 '2021-06-04',
 '2021-08-06',
 '2021-09-03',
 '2021-10-08',
 '2021-10-08',
 '2021-10-08',
 '2021-10-08',
 '2021-10-08',
 '2021-10-08',
 '2021-11-05',
 '2022-02-04',
 '2022-04-01',
 '2022-06-03',
 '2022-07-08',
 '2022-07-08',
 '2022-07-08',
 '2022-07-08',
 '2022-07-08',
 '2022-07-08',
 '2022-08-05',
 '2022-09-02',
 '2022-11-04',
 '2023-02-03',
 '2023-04-07',
 '2023-06-02',
 '2023-08-04',
 '2023-09-01',
 '2023-11-03',
 '2023-12-08',
 '2023-12-08',
 '2023-12-08',
 '2023-12-08',
 '2023-12-08',
 '2023-12-08',
 '2024-02-02',
 '2024-03-08',
 '2024-03-08',
 '2024-03-08',
 '2024-03-08',
 '2024-03-08',
 '2024-03-

In [151]:
len(lista_atualizada)

72